# 第5章: 係り受け解析

日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

KNP使うときのコマンド

`juman < ai.ja.txt | knp -simple -anaphora > ai.ja.txt.parsed`

In [1]:
import os
from typing import List, Dict

In [2]:
input_path = os.path.join(os.getcwd(), '../data/ai.ja.txt.parsed')

In [3]:
# 読み込み
def get_raw(path: str) -> str:
    with open(path, mode='r') as f:
        return f.read()

In [4]:
# 行ごとに読み込み
def get_lines(path: str) -> List[str]:
    with open(path, mode='r') as f:
        s = [s.strip() for s in f.readlines()]
    return s

In [5]:
# 全文
raw = get_raw(input_path)

In [6]:
# 文リスト
sentences = raw.split('EOS\n')

## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [7]:
# 形態素を表すクラス
class Morph():
    def __init__(self, morph_dict: dict):
        self.surface = morph_dict['surface'] # 表層形
        self.base = morph_dict['base'] # 基本形
        self.pos = morph_dict['pos'] # 品詞
        self.pos1 = morph_dict['pos1'] # 品詞細分類1

Morph({'surface': 'a', 'base': 'b', 'pos': 'c', 'pos1': 'd'})

In [8]:
# 入力形態素から始まる行を渡すと、表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）の辞書を返す
def get_surface_base_pos_pos1(text: str) -> dict:
    morp = text.split(' ')
    surface = morp[0]
    base = morp[2]
    pos = morp[3]
    pos1 = morp[5]
    return {
        'surface': surface,
        'base': base,
        'pos': pos,
        'pos1': pos1
    }

tes = 'a b c d e f'
expect = {
    'surface': 'a',
    'base': 'c',
    'pos': 'd',
    'pos1': 'f'
}
get_surface_base_pos_pos1(tes) == expect

True

In [9]:
# 1文を渡すと、形態素リストを返す
def get_sentence_morphs(sentence: str) -> List:
    # 改行で分ける
    # 長さ0の文字列は除外(末尾の改行によって空白になる要素)
    lines = [l for l in sentence.split('\n') if len(l) != 0]
    ret_list = []
    # 行ごとに処理
    for l in lines:
        # #, *, +で始まる行は形態素でない情報なのでパス
        if l[:1] == '#' or l[:1] == '*' or l[:1] == '+':
            continue
        # その他の（入力形態素から始まる）場合
        else:
            # 形態素を辞書型で取得
            morph_dict = get_surface_base_pos_pos1(l)
            # インスタンス作ってリストに入れる
            ret_list.append(Morph(morph_dict))
    return ret_list

tes = sentences[2]
''.join([m.surface for m in get_sentence_morphs(tes)])

'人工知能（じんこうちのう、、AI〈エーアイ〉）とは、「『計算（）』という概念と『コンピュータ（）』という道具を用いて『知能』を研究する計算機科学（）の一分野」を指す語。「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピューターに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。'

In [10]:
# 出力
output = [get_sentence_morphs(s) for s in sentences]

In [11]:
# 表示
[(o.surface, o.base, o.pos, o.pos1) for o in output[2]]

[('人工', '人工', '名詞', '普通名詞'),
 ('知能', '知能', '名詞', '普通名詞'),
 ('（', '（', '特殊', '括弧始'),
 ('じんこう', 'じんこう', '名詞', '普通名詞'),
 ('ちのう', 'ちのう', '名詞', '普通名詞'),
 ('、', '、', '特殊', '読点'),
 ('、', '、', '特殊', '読点'),
 ('AI', 'AI', '名詞', '普通名詞'),
 ('〈', '〈', '特殊', '括弧始'),
 ('エーアイ', 'エーアイ', '名詞', '普通名詞'),
 ('〉', '〉', '特殊', '括弧終'),
 ('）', '）', '特殊', '括弧終'),
 ('と', 'と', '助詞', '格助詞'),
 ('は', 'は', '助詞', '副助詞'),
 ('、', '、', '特殊', '読点'),
 ('「', '「', '特殊', '括弧始'),
 ('『', '『', '特殊', '括弧始'),
 ('計算', '計算', '名詞', 'サ変名詞'),
 ('（）', '（）', '名詞', '普通名詞'),
 ('』', '』', '特殊', '括弧終'),
 ('と', 'と', '助詞', '格助詞'),
 ('いう', 'いう', '動詞', '*'),
 ('概念', '概念', '名詞', '普通名詞'),
 ('と', 'と', '助詞', '格助詞'),
 ('『', '『', '特殊', '括弧始'),
 ('コンピュータ', 'コンピュータ', '名詞', '普通名詞'),
 ('（）', '（）', '名詞', '普通名詞'),
 ('』', '』', '特殊', '括弧終'),
 ('と', 'と', '助詞', '格助詞'),
 ('いう', 'いう', '動詞', '*'),
 ('道具', '道具', '名詞', '普通名詞'),
 ('を', 'を', '助詞', '格助詞'),
 ('用いて', '用いる', '動詞', '*'),
 ('『', '『', '特殊', '括弧始'),
 ('知能', '知能', '名詞', '普通名詞'),
 ('』', '』', '特殊', '括弧終'),
 ('を', '

## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [12]:
# 文節を表すクラス
class Chunk():
    def __init__(self, chunk_dict: dict):
        self.morphs = chunk_dict['morphs']
        self.dst = chunk_dict['dst']
        self.srcs = chunk_dict['srcs']

In [13]:
import re
# 文節を渡すと、係り先と形態素リストを返す
def get_chunk_morphs(sentence: str) -> List:
    chunk = {}
    lines = sentence.split('\n')
    # 係り先
    chunk['dst'] = int(re.match(r'-*[0-9]+', lines[0]).group())
    # 形態素リスト
    chunk['morphs'] = get_sentence_morphs('\n'.join(lines[1:]))
    return chunk

In [14]:
# 1文を渡すと、文節リストを返す
def get_sentence_chunks(sentence: str) -> List:
    chunks_raw = sentence.split('\n* ')[1:] # 先頭は、#から始まる解析文の情報なので除外
    chunks = {}
    for i, c in enumerate(chunks_raw):
        chunks[i] = get_chunk_morphs(c)
        chunks[i]['srcs'] = []
    for i, c in chunks.items():
        if c['dst'] != -1:
            chunks[c['dst']]['srcs'].append(i)
    return [Chunk(c) for c in chunks.values()]

In [15]:
# 表示
[(c.dst, [m.surface for m in c.morphs]) for c in get_sentence_chunks(sentences[2])]

[(3, ['人工', '知能']),
 (3, ['（', 'じんこう', 'ちのう', '、', '、']),
 (3, ['AI']),
 (15, ['〈', 'エーアイ', '〉', '）', 'と', 'は', '、']),
 (5, ['「', '『', '計算', '（）', '』', 'と']),
 (6, ['いう']),
 (9, ['概念', 'と']),
 (8, ['『', 'コンピュータ', '（）', '』', 'と']),
 (9, ['いう']),
 (10, ['道具', 'を']),
 (12, ['用いて']),
 (12, ['『', '知能', '』', 'を']),
 (13, ['研究', 'する']),
 (14, ['計算', '機', '科学', '（）', 'の']),
 (15, ['一', '分野', '」', 'を']),
 (16, ['指す']),
 (36, ['語', '。']),
 (20, ['「', '言語', 'の']),
 (19, ['理解', 'や']),
 (20, ['推論', '、']),
 (21, ['問題', '解決', 'など', 'の']),
 (25, ['知的', '行動', 'を']),
 (23, ['人間', 'に']),
 (25, ['代わって']),
 (25, ['コンピューター', 'に']),
 (26, ['行わ', 'せる']),
 (35, ['技術', '」', '、', 'または', '、']),
 (28, ['「', '計算', '機']),
 (29, ['（', 'コンピュータ', '）', 'に']),
 (31, ['よる']),
 (31, ['知的な']),
 (33, ['情報', '処理', 'システム', 'の']),
 (33, ['設計', 'や']),
 (34, ['実現', 'に']),
 (35, ['関する']),
 (36, ['研究', '分野', '」', 'と', 'も']),
 (-1, ['さ', 'れる', '。'])]

In [16]:
# 出力
output = [get_sentence_chunks(s) for s in sentences]
output = [o for o in output if len(o) != 0] # 長さ0の要素は除外

## 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [17]:
def show_dst_srcs(chunks):
    for c in chunks:
        print(''.join([m.surface for m in c.morphs if m.pos != '特殊']),
              '\t',
              ''.join([m.surface for m in chunks[c.dst].morphs  if m.pos != '特殊']))

show_dst_srcs(output[1])

人工知能 	 エーアイとは
じんこうちのう 	 エーアイとは
AI 	 エーアイとは
エーアイとは 	 指す
計算（）と 	 いう
いう 	 概念と
概念と 	 道具を
コンピュータ（）と 	 いう
いう 	 道具を
道具を 	 用いて
用いて 	 研究する
知能を 	 研究する
研究する 	 計算機科学（）の
計算機科学（）の 	 一分野を
一分野を 	 指す
指す 	 語
語 	 される
言語の 	 問題解決などの
理解や 	 推論
推論 	 問題解決などの
問題解決などの 	 知的行動を
知的行動を 	 行わせる
人間に 	 代わって
代わって 	 行わせる
コンピューターに 	 行わせる
行わせる 	 技術または
技術または 	 研究分野とも
計算機 	 コンピュータに
コンピュータに 	 よる
よる 	 情報処理システムの
知的な 	 情報処理システムの
情報処理システムの 	 実現に
設計や 	 実現に
実現に 	 関する
関する 	 研究分野とも
研究分野とも 	 される
される 	 される


In [18]:
# # 解答
# for chunks in output:
#     show_dst_srcs(chunks)

## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [19]:
def show_noun_verb(chunks):
    for c in chunks:
        # 係り元
        morphs1 = c.morphs
        # 係り先
        morphs2 = chunks[c.dst].morphs
        # 係り元に名詞がなければパス
        if '名詞' not in [m.pos for m in morphs1]:
            continue
        # 係り先に動詞がなければパス
        if '動詞' not in [m.pos for m in morphs2]:
            continue
        print(''.join([m.surface for m in morphs1 if m.pos != '特殊']),
              '\t',
              ''.join([m.surface for m in morphs2 if m.pos != '特殊']))

show_noun_verb(output[1])

エーアイとは 	 指す
計算（）と 	 いう
コンピュータ（）と 	 いう
道具を 	 用いて
知能を 	 研究する
一分野を 	 指す
語 	 される
知的行動を 	 行わせる
人間に 	 代わって
コンピューターに 	 行わせる
コンピュータに 	 よる
実現に 	 関する
研究分野とも 	 される


In [20]:
# # 解答
# for chunks in output:
#     show_noun_verb(chunks)

## 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

In [21]:
from graphviz import Digraph

In [22]:
# 有向グラフ
dg = Digraph(format='png')
# 日本語表示
dg.attr('node', shape='box', fontname='MS Gothic')
# 中身
dg.node('あ')
dg.node('2')
dg.node('3')
dg.edge('あ', '2')  # 1 -> 2
dg.edge('2', '3')  # 2 -> 3
dg.edge('3', 'あ')  # 3 -> 1
dg.format = 'png'
dg.render(view = True)

'Digraph.gv.png'

In [23]:
def show_digraph(chunks, file_name: str = 'temp'):
    # 有向グラフ
    dg = Digraph(format='png')
    # 日本語表示
    dg.attr('node', shape='box', fontname='MS Gothic')
    # 中身
    for i, c in enumerate(chunks):
        # 係り元
        morphs1 = c.morphs
        node1 = str(i) + '_' + ''.join([m.surface for m in morphs1 if m.pos != '特殊'])
        # ノード
        dg.node(node1)
        # 係り先
        if c.dst != -1: # 最後の文節は係り先がないので除外
            morphs2 = chunks[c.dst].morphs
            node2 = str(c.dst) + '_' + ''.join([m.surface for m in morphs2 if m.pos != '特殊'])
            dg.edge(node1, node2)
    dg.format = 'png'
    dg.render(file_name)

show_digraph(output[3])

In [24]:
# # 解答
# for i, chunks in enumerate(output):
#     print(i)
#     show_digraph(chunks, str(i))

## 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ．

* 動詞を含む文節において，最左の動詞の基本形を述語とする
* 述語に係る助詞を格とする
* 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

* コーパス中で頻出する述語と格パターンの組み合わせ
* 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [25]:
def get_predicate_postpositional(chunks):
    ret_list = []
    for c in chunks:
        # 述語
        morphs1 = c.morphs
        # 述語候補に助詞がなければパス
        if '動詞' not in [m.pos for m in morphs1]:
            continue
        # 動詞を含む文節において，最左の動詞の基本形を述語とする
        for m in morphs1:
            if m.pos == '動詞':
                predicate = m.base
        
        # 格
        morphs2 = [chunks[s].morphs for s in c.srcs]
        # 助詞を入れるリスト
        postpositional = []
        # 全ての係り元を調べる
        for m2 in morphs2:
            # 係り元の形態をを調べる
            for m in m2:
                # 助詞をストック
                if m.pos == '助詞':
                    postpositional.append(m.surface)
        # 述語、格を保存
        ret_list.append([
            predicate,
            ' '.join(sorted(postpositional)), # スペース区切りで辞書順に並べる
        ])
    return ret_list

get_predicate_postpositional(output[1])

[['いう', 'と'],
 ['いう', 'と'],
 ['用いる', 'を'],
 ['する', 'を'],
 ['指す', 'と は を'],
 ['代わる', 'に'],
 ['行う', 'に を'],
 ['よる', 'に'],
 ['関する', 'に'],
 ['する', 'と も']]

In [26]:
# 出力
with open('45.txt', 'w') as f:
    for chunks in output:
        pred_posts = get_predicate_postpositional(chunks)
        for pred, post  in pred_posts:
            print(pred + '\t' + post, file=f)

In [29]:
# コーパス中で頻出する述語と格パターンの組み合わせ
# TOP10表示
!cut -f 1,2 45.txt | sort | uniq -c | sort -nr | head -10

     29 する	を
     24 いう	と
     22 する	と
     20 する	が
     19 よる	に
     13 する	
     10 する	は を
     10 する	で を
      8 する	に
      5 する	に は を


In [61]:
# 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順）
!cut -f 1,2 45.txt | sort | uniq -c | sort -nr | grep -e [0-9]*\s"行う" -e [0-9]*\s"なる" -e [0-9]*\s"*与える"

      3 行う	を
      3 行う	に を
      2 行う	は を
      2 なる	が に
      2 なる	が と
      2 なる	
      1 与える	が など に
      1 行う	まで を
      1 行う	に は は は
      1 行う	に
      1 行う	から は
      1 なる	に
      1 なる	と に は も
      1 なる	と
      1 なる	で に
      1 なる	が で に は は
      1 なる	が が と
      1 なる	から で と に は まで


## 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

* 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
* 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

In [82]:
def get_predicate_postpositional_term(chunks):
    ret_list = []
    for c in chunks:
        # 述語
        morphs1 = c.morphs
        # 述語候補に助詞がなければパス
        if '動詞' not in [m.pos for m in morphs1]:
            continue
        # 動詞を含む文節において，最左の動詞の基本形を述語とする
        for m in morphs1:
            if m.pos == '動詞':
                predicate = m.base
        
        # 格、項
        morphs2 = [chunks[s].morphs for s in c.srcs]
        # 格、項を入れるリスト
        postpositional = []
        # 全ての係り元を調べる
        for m2 in morphs2:
            # 助詞がなければパス
            if '助詞' not in [m.pos for m in m2]:
                continue
            # 助詞を探す
            for m in m2:
                if m.pos == '助詞':
                    post = m.surface
            # 格、項を保存
            postpositional.append([
                post, # 格
                ''.join([m.surface for m in m2 if m.pos != '特殊']) # 項
            ])
        # ソート
        postpositional = [p.split(' ') for p in sorted(
            [' '.join(p) for p in postpositional]
        )]
        ret_list.append([
            predicate,
            ' '.join(p[0] for p in postpositional), # スペース区切りで格を表示
            ' '.join(p[1] for p in postpositional), # スペース区切りで項を表示
        ])
    return ret_list

get_predicate_postpositional_term(output[1])

[['いう', 'と', '計算（）と'],
 ['いう', 'と', 'コンピュータ（）と'],
 ['用いる', 'を', '道具を'],
 ['する', 'を', '知能を'],
 ['指す', 'は を', 'エーアイとは 一分野を'],
 ['代わる', 'に', '人間に'],
 ['行う', 'に を', 'コンピューターに 知的行動を'],
 ['よる', 'に', 'コンピュータに'],
 ['関する', 'に', '実現に'],
 ['する', 'も', '研究分野とも']]

In [84]:
# 出力
with open('46.txt', 'w') as f:
    for chunks in output:
        pred_posts = get_predicate_postpositional_term(chunks)
        for pred, post, term  in pred_posts:
            print(pred + '\t' + post + '\t' + term, file=f)

## 47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

* 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
* 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
* 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
* 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）